# Named Entity Recognition

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/KennethEnevoldsen/DaCy/blob/master/docs/performance.ner.ipynb)


This page examines the performance of competing models for Danish named entity recognition over multiple datasets. Performance is not limited to 
accuracy, but also includes domain generalization, biases and robustness. This page is also a notebook, which open and replicate the results.

## State-of-the-Art comparison
To our knowledge there exists three datasets for Danish named entity recognition;

1) DaNE {cite}`hvingelby2020dane`, which uses the simple annotation scheme of CoNLL 2003 {cite}`missing` with the entities; *person*, *location*, *organization*, and *miscellaneus*.
2) DANSK {cite}`missing`, which uses the extensive annotation scheme similar to that of OntoNotes 5.0 {cite}`missing` including more that 16 entity types.
3) and DAN+ {cite}`missing`, which also uses the annotation scheme of CoNLL 2003, but allows for nested entities for instance *Aarhus Universitet*, where *Aarhus* is a location and *Aarhus Universitet* is an organization.

In this comparison we will be examing performance on DaNE and DANSK, but as no known models have been trained on Danish nested entities, we will not be comparing performance on DAN+.


```{admonition} Measuring Performance
Typically when measuring performance on these benchmark it is normal to feed the model the gold standard tokens. While this allows for easier comparisons of modules and architectures, it inflates the performance metrics. Further, it does not proberly reflect what you are really interested in:
*the performance you can expect when you apply the model to data of a similar type*. Therefore we estimate the model is given no prior knowledge of the data, and only the raw text is fed to the model. Thus the performance metrics might be slightly different compared to e.g. DaNLP.
```

### DaNE: Simple Named Entity Recognition
As already stated DaNE uses an extraction from the CoNLL 2003 dataset, which is as follows {cite}`hvingelby2020dane`:




| Entity | Description |
|--------------|-------------|
| LOC          | includes locations like cities, roads and mountains, as well as both public and commercial places like specific buildings or meeting points, but also abstract places. |
| PERSON | consists of names of people, fictional characters, and animals. The names includes aliases. |
| ORG | can be summarized as all sorts of organizations and collections of people, ranging from companies, brands, political movements, governmental bodies and clubs. |
| MISC | is a broad category of e.g. events, languages, titles and religions, but this tag also includes words derived from one of the four tags as well as words for which one part is from one of the three other tags. |

Here is an example from the dataset:

In [1]:
import spacy
from spacy.tokens import Span
from spacy import displacy

text = """To kendte russiske historikere Andronik Mirganjan og Igor Klamkin tror ikke, at Rusland kan udvikles uden en "jernnæve"."""
nlp = spacy.blank("da")
doc = nlp(text)
doc.ents = [  # type: ignore
    Span(doc, 2, 3, label="MISC"),
    Span(doc, 4, 6, label="PERSON"),
    Span(doc, 7, 9, label="PERSON"),
    Span(doc, 13, 14, label="LOC"),
]

displacy.render(doc, style="ent")

The table below shows the performance of Danish language processing pipelines scored on the DaNE test set. The best scores in each category are highlighted with bold and the second best is underlined.

In [2]:
from evaluation.models import MODELS
from evaluation.utils import apply_models, bootstrap, compute_mean_and_ci
from multiprocessing import Pool

In [3]:
dane = {}
with Pool(8) as p:
    out = p.starmap(
        apply_models,
        [
            (mdl_name, model_getter, "dane", ["test"])
            for mdl_name, model_getter in MODELS.items()
        ],
    )
    for mdl_name, mdl_results in zip(MODELS.keys(), out):  # type: ignore
        dane[mdl_name] = mdl_results["test"]

dane (test): Loading prediction for da_dacy_small_ner_fine_grained-0.1.0
dane (test): Loading prediction for da_dacy_small_trf-0.2.0
dane (test): Loading prediction for da_dacy_medium_ner_fine_grained-0.1.0
dane (test): Loading prediction for da_dacy_large_trf-0.2.0
dane (test): Loading prediction for da_dacy_medium_trf-0.2.0
dane (test): Loading prediction for da_dacy_large_ner_fine_grained-0.1.0
dane (test): Loading prediction for saattrupdan/nbailab-base-ner-scandi
dane (test): Loading prediction for alexandrainst/da-ner-base
dane (test): Loading prediction for da_core_news_trf-3.5.0
dane (test): Loading prediction for da_core_news_lg-3.5.0
dane (test): Loading prediction for da_core_news_md-3.5.0
dane (test): Loading prediction for da_core_news_sm-3.5.0


In [ ]:
from spacy.training import Example
import pandas as pd
from evaluation.utils import create_dataframe

def highlight_max(s: pd.Series) -> list:
    """Highlight the maximum in a Series with bold text."""
    # convert to str for comparison
    s = s.astype(str)
    is_max = s == s.max()
    return ["font-weight: bold" if v else "" for v in is_max]


def underline_second_max(s: pd.Series) -> list:
    """Underline the second maximum in a Series."""
    is_second_max = s == s.sort_values(ascending=False).iloc[1]
    return ["text-decoration: underline" if v else "" for v in is_second_max]


def create_table(
    df: pd.DataFrame,
    caption="F1 score with 95% confidence interval calculated using bootstrapping with 100 samples.",
):
    # replace index with range
    df.index = range(len(df))  # type: ignore

    col_names = [("", "Models")] + [("F1", col) for col in df.columns[1:]]
    super_header = pd.MultiIndex.from_tuples(col_names)
    df.columns = super_header

    s = df.style.apply(highlight_max, axis=0, subset=df.columns[1:])
    s = s.apply(underline_second_max, axis=0, subset=df.columns[1:])

    # Add a caption
    s = s.set_caption(caption)

    # Center the header and left align the model names
    s = s.set_properties(subset=df.columns[1:], **{"text-align": "right"})

    super_header_style = [
        {"selector": ".level0", "props": [("text-align", "center")]},
        {"selector": ".col_heading", "props": [("text-align", "center")]},
    ]
    # Apply the CSS style to the styler
    s = s.set_table_styles(super_header_style)  # type: ignore
    s = s.set_properties(subset=[("", "Models")], **{"text-align": "left"})
    # remove the index
    s = s.hide(axis="index")
    return s

In [ ]:
with Pool(8) as p:
    tables = p.starmap(
        create_dataframe,
        [(dane[mdl]["examples"], mdl, 1, 500) for mdl in dane if "fine_grained" not in mdl],
    )

In [ ]:
df = pd.concat(tables)
# sort columns
df = df[["Models", "Average", "Location", "Person", "Organization", "Misc."]]
create_table(
    df,
    "F1 score with 95% confidence interval calculated using bootstrapping with 500 samples.",
)

It is worth mentioning that while the `da_dacy_large_trf-0.2.0` and `saattrupdan/nbailab-base-ner-scandi` performs similarly they have their independent strength and weaknesses. The large DaCy model is a multi-task model performing named-entity recognition as only one of its many tasks and thus if you wish to use one of those we would recommend that model. On the other hand the `nbailab-base-ner-scandi` is trained on multiple Scandinavian languages and thus might be ideal if your dataset might contain these languages as well. `saattrupdan/nbailab-base-ner-scandi` is available in DaCy using `nlp.add_pipe("dacy/ner")`.

```{admonition} You are missing a model
:note:

These tables are continually updated and thus we try to limit the number of models to only the most relevant Danish models. Therefore models like Polyglot with strict requirements and consistently worse performance are excluded. If you want to see a specific model, please open an issue on GitHub.
```




### DANSK: Fine-grained Named Entity Recognition

DANSK is annotated from the Danish Gigaword Corpus {cite}`missing` and a wide variety of domains including conversational, legal, news, social media, web content,  wiki's and Books. Dansk follows includes the following labels:





|  Entity        |             Description                                         |
| -------- | ---------------------------------------------------- |
| PERSON   | People, including fictional                          |
| NORP     | Nationalities or religious or political groups       |
| FACILITY | Building, airports, highways, bridges, etc.          |
| ORGANIZATION | Companies, agencies, institutions, etc.              |
| GPE      | Countries, cities, states.                           |
| LOCATION | Non-GPE locations, mountain ranges, bodies of water  |
| PRODUCT  | Vehicles, weapons, foods, etc. (not services)        |
| EVENT    | Named hurricanes, battles, wars, sports events, etc. |
| WORK OF ART | Titles of books, songs, etc.                         |
| LAW      | Named documents made into laws                       |
| LANGUAGE | Any named language                                   |

As well as annotation for the following concepts:

|   Entity       |   Description                                         |
| -------- | ------------------------------------------- |
| DATE     | Absolute or relative dates or periods       |
| TIME     | Times smaller than a day                    |
| PERCENT  | Percentage (including "*"%)                |
| MONEY    | Monetary values, including unit             |
| QUANTITY | Measurements, as of weight or distance      |
| ORDINAL  | "first", "second"                           |
| CARDINAL | Numerals that do no fall under another type |


We have here opted to create an interactive chart over a table as with the number of labels it quickly becomes unruly. The chart is interactive and you can select the label you want to compare the models on. You can also hover over the dots the see the exact values.

In [ ]:
dansk = {}
for mdl_name, model_getter in MODELS.items():
    mdl_results = apply_models(
        mdl_name, model_getter, dataset="dansk", splits=["train", "dev", "test"]
    )
    dansk[mdl_name] = mdl_results

In [ ]:
# tables = []
# for mdl in dansk:
#     # only fine-grained models are trained on DANSK
#     if "fine_grained" not in mdl:
#         continue
#     tables.append(
#         create_dataframe(dansk[mdl]["test"]["examples"], mdl, n_rep=100, n_samples=2000)
#     )

with Pool(8) as p:
    tables = p.starmap(
        create_dataframe,
        [(dansk[mdl]["test"]["examples"], mdl, 1, 100, 2000) for mdl in dansk if "fine_grained" in mdl],
    )


In [ ]:
import altair as alt


def create_dansk_viz(df: pd.DataFrame):
    plot_df = df.melt(
        id_vars=["Models"],
        var_name="Label",
        value_name="F1 string",
    )

    # Convert the score value to a float
    plot_df["F1"] = plot_df["F1 string"].apply(
        lambda x: float(x.split()[0]) if not isinstance(x, float) else x
    )
    plot_df["CI Lower"] = plot_df["F1 string"].apply(
        lambda x: float(x.split("(")[1].split(",")[0])
    )
    plot_df["CI Upper"] = plot_df["F1 string"].apply(
        lambda x: float(x.split(",")[1].split(")")[0])
    )

    selection = alt.selection_point(
        fields=["Label"],
        bind="legend",
        value=[{"Label": "Average"}],
    )

    base = (
        alt.Chart(plot_df)
        .mark_point(filled=True, size=100)
        .encode(
            x=alt.X("F1", title="F1"),
            y="Models",
            color="Label",
            tooltip=[
                "Models",
                "Label",
                alt.Tooltip("F1 string", title="F1"),
            ],
            opacity=alt.condition(selection, alt.value(1), alt.value(0.0)),
            # only show the tooltip when when the label is selected
        )
    )
    error_bars = (
        alt.Chart(plot_df)
        .mark_errorbar(ticks=False)
        .encode(
            x=alt.X("CI Lower", title="F1"),
            x2="CI Upper",
            y="Models",
            color="Label",
            opacity=alt.condition(selection, alt.value(1), alt.value(0.0)),
        )
    )

    chart = base + error_bars

    return chart.add_params(selection).properties(width=800, height=400)

In [ ]:
dansk_df = pd.concat(tables)
create_dansk_viz(dansk_df)

In [ ]:
_df = dansk_df
_df = _df.set_index("Models")
ent_columns = sorted(
    [
        "Event",
        "Organization",
        "Language",
        "Person",
        "Ordinal",
        "NORP",
        "Work of Art",
        "Facility",
        "Law",
        "Location",
        "Product",
        "GPE",
    ]
)
non_ent_columns = sorted(["Cardinal", "Date", "Money", "Percent", "Quantity", "Time"])
columns_to_keep = ent_columns + non_ent_columns + ["Average"]

_df = _df[columns_to_keep]

In [ ]:
table = _df.T
# iidx = pd.MultiIndex.from_product([ent_columns, non_ent_columns, ["Average"]], names=["Entity", "Non-Entity", ""])
iidx = pd.MultiIndex.from_arrays(
    [
        ["Entities"] * len(ent_columns)
        + ["Non-Entities"] * len(non_ent_columns)
        + ["Average"],
        ent_columns + non_ent_columns + ["Average"],
    ]
)
table.index = iidx

mdl_names = ["Large 0.1.0", "Medium 0.1.0", "Small 0.1.0"]
header = pd.MultiIndex.from_arrays(
    [["Fine-grained Models"] * len(mdl_names), mdl_names]
)
table.columns = header

In [ ]:
# convert to latex using styler
style = table.style.format_index(escape="latex", axis=1).format_index(
    escape="latex", axis=0
)


# highlight the maximum
def italicize_second_max(s: pd.Series) -> list:
    """Italicize the second maximum in a Series."""
    is_second_max = s == s.sort_values(ascending=False).iloc[1]
    # check if the second maximum is the same as the maximum
    same_as_max = s == s.max()

    if same_as_max.sum() > 1:
        # if there are more than one maximum, don't italicize
        return ["font-style: normal" for v in is_second_max]
    return ["font-style: italic" if v else "" for v in is_second_max]


style = style.apply(highlight_max, axis=1)
# style = style.apply(underline_second_max, axis=1)
style = style.apply(italicize_second_max, axis=1)

# apply the CSS style
super_header_style = [
    {"selector": ".level0", "props": [("text-align", "center")]},
    {"selector": ".col_heading", "props": [("text-align", "center")]},
]
style = style.set_table_styles(super_header_style)


# add caption
caption = "F1 score with 95% confidence interval calculated using bootstrapping with 100 samples."
style = style.set_caption(caption)
style

# latex = style.to_latex(
#         hrules=True,
#         convert_css=True,
#     )

# print(latex)

## Domain Generalization
For the domains generalization benchmark we utilize the [DANSK](https://huggingface.co/datasets/chcaa/DANSK) dataset {cite}`missing`. This dataset is annotated across many different domains including fiction, web content, social media, wikis, news, legal and conversational data.
As some models are trained on DANSK (`da_dacy_{size}_ner_fine_grained-{version}`) these models are tested on the test set using all of the
labels. 

In [ ]:
from evaluation.utils import evaluate_generalization

In [ ]:
tables = []
for mdl_name in dansk:
    if "fine_grained" not in mdl_name:
        continue

    table = evaluate_generalization(examples=dansk[mdl_name]["test"]["examples"], mdl_name=mdl_name, n_rep=100, n_samples=1000)
    tables.append(table)


In [ ]:
df = pd.concat(tables)
df = df[df["Domain"] != "dannet"]
df = df[df["Domain"].notnull()]

In [ ]:
# create altair viz
selection = alt.selection_point(
    fields=["Domain"],
    bind="legend",
    value=[{"Domain": "All"}],
)
bind_checkbox = alt.binding_checkbox(
    name="Scale point size by number of documents: ",
)
param_checkbox = alt.param(bind=bind_checkbox)

sort_order = list(dansk.keys())

base = (
    alt.Chart(df)
    .mark_point(filled=True)
    .encode(
        x=alt.X("Average", title="F1", scale=alt.Scale(domain=[0.0, 1.0])),
        y=alt.Y("Model", sort=sort_order),
        color="Domain",
        size=alt.condition(
            param_checkbox, "Number of docs", alt.value(100), legend=None
        ),
        tooltip=[
            "Model",
            "Domain",
            "Average F1",
        ],
        opacity=alt.condition(selection, alt.value(1), alt.value(0.0)),
    )
)
error_bars = (
    alt.Chart(df)
    .mark_errorbar(ticks=False)
    .encode(
        x=alt.X("Average Lower CI", title="F1"),
        x2="Average Upper CI",
        y=alt.Y("Model", sort=sort_order),
        color="Domain",
        opacity=alt.condition(selection, alt.value(1), alt.value(0.0)),
    )
)

chart = base + error_bars

chart = chart.add_params(selection, param_checkbox).properties(
    width=800,
    height=400,
    title="DANSK test set performance",
)


chart


### Domain generalization using CoNLL-2003 format
To test the generalization we here convert the annotations to the CoNLL-2003 format using the labels `Person`, `Location`, `Organization`. As CoNLL-2003, `Location` includes cities, roads, mountains, abstract places, specific buildings, and meeting points. Thus the `GPE` (geo-political entity) were converted to `Location`. The `MISC` category in CoNLL-2003 is a diverse category meant to denote all names not in other categories (encapsulating both e.g. events and adjectives such as ”2004 World Cup” and ”Italian”), and is therefore not included.

In [ ]:
from evaluation.utils import convert_to_conll_2003, create_row_conll2003

In [ ]:
tables = []
for mdl_name in dansk:
    if "fine_grained" in mdl_name:
        continue
    examples = dansk[mdl_name]["test"]["examples"]
    examples += dansk[mdl_name]["dev"]["examples"]
    examples += dansk[mdl_name]["train"]["examples"]

    
    examples = convert_to_conll_2003(examples)
    table = evaluate_generalization(mdl_name, examples, n_rep=100, n_samples=1000, create_row_fn=create_row_conll2003)
    tables.append(table)

tables = pd.concat(tables, axis=0)

In [ ]:
df = tables
df = df[df["Domain"] != "dannet"]   # type: ignore
df = df[df["Domain"].notnull()]

In [ ]:
# create altair viz
selection = alt.selection_point(
    fields=["Domain"],
    bind="legend",
    value=[{"Domain": "All"}],
)
bind_checkbox = alt.binding_checkbox(
    name="Scale point size by number of documents: ",
)
param_checkbox = alt.param(bind=bind_checkbox)

sort_order = list(dansk.keys())

base = (
    alt.Chart(df)
    .mark_point(filled=True)
    .encode(
        x=alt.X("Average", title="F1", scale=alt.Scale(domain=[0.0, 1.0])),
        y=alt.Y("Model", sort=sort_order),
        color="Domain",
        size=alt.condition(
            param_checkbox, "Number of docs", alt.value(100), legend=None
        ),
        tooltip=[
            "Model",
            "Domain",
            "Average F1",
            "Person F1",
            "Location F1",
            "Organization F1",
        ],
        opacity=alt.condition(selection, alt.value(1), alt.value(0.0)),
    )
)
error_bars = (
    alt.Chart(df)
    .mark_errorbar(ticks=False)
    .encode(
        x=alt.X("Average Lower CI", title="F1"),
        x2="Average Upper CI",
        y=alt.Y("Model", sort=sort_order),
        color="Domain",
        opacity=alt.condition(selection, alt.value(1), alt.value(0.0)),
    )
)

chart = base + error_bars

chart = chart.add_params(selection, param_checkbox).properties(
    width=800,
    height=400,
    title="Generalization to Unseen Domains",
)


chart

## Biases

To examine the biases in Danish models we use augmentation to replace names in the Danish dataset DaNE {cite}`hvingelby2020dane`, this approach
is similar to that introduced in the initial DaCy paper {cite}`enevoldsen2021dacy`.

Here is a short example of how the augmentation might look like:


````{admonition} Example

```{admonition} Original
:class: note


Peter Schmeichel mener også, at det danske landshold anno 2021 tilhører verdenstoppen og kan vinde den kommende kamp mod England.
```

```{admonition} Female name augmentation
:class: important

Anne Østergaard mener også, at det danske landshold anno 2021 tilhører verdenstoppen og kan vinde den kommende kamp mod England.
```
````


In [ ]:
gbiases = {}
for mdl_name, model_getter in MODELS.items():
    if "fine_grained" in mdl_name:
        continue
    mdl_results = apply_models(
        mdl_name, model_getter, dataset="gender_bias_dane", splits=["test"]
    )
    gbiases[mdl_name] = mdl_results

In [ ]:
from collections import defaultdict
def augmentation_specific_examples(examples):
    aug_group = defaultdict(list)
    for example in examples:
        aug_name = example.y._.meta["augmenter"]
        aug_group[aug_name].append(example)
    return aug_group

In [ ]:
tables = []
for mdl in gbiases:
    print(mdl)
    examples = gbiases[mdl]["test"]["examples"]

    aug_group = augmentation_specific_examples(examples)
    for aug_name, _examples in aug_group.items():
        _examples = convert_to_conll_2003(_examples) # also removes misc.
        table = create_dataframe(_examples, mdl, n_rep=100, n_samples=1000)
        table["Augmentation"] = aug_name
        tables.append(table)

In [ ]:
df = pd.concat(tables)

In [ ]:
# create the table
def create_table(df, model_order: list[str]):

    table_df = df[["Models", "Augmentation", "Average"]]

    table_df = table_df.pivot(index="Models", columns="Augmentation", values="Average")

    # order the models column
    table_df = table_df.reindex(model_order)


    # create augmentation superheader

    aug_superheader = []# [("", "Models")]
    for aug_name in table_df.columns:
        aug_superheader.append(("Augmentation", aug_name))

    aug_superheader = pd.MultiIndex.from_tuples(aug_superheader)
    table_df.columns = aug_superheader
    df = table_df.reset_index()
    s = df.style.apply(highlight_max, axis=0, subset=df.columns[1:])
    s = s.apply(underline_second_max, axis=0, subset=df.columns[1:])

    # Add a caption
    s = s.set_caption("F1 score for each augmentation with 95% confidence interval calculated over 100 repetitions")

    # Center the header and left align the model names
    s = s.set_properties(subset=df.columns[1:], **{"text-align": "right"})

    super_header_style = [
        {"selector": ".level0", "props": [("text-align", "center")]},
        {"selector": ".col_heading", "props": [("text-align", "center")]},
    ]
    # Apply the CSS style to the styler
    s = s.set_table_styles(super_header_style)  # type: ignore
    # s = s.set_properties(subset=[("", "Models")],
    #                       **{"text-align": "left"})
    # remove the index
    s = s.hide(axis="index")
    return s



In [ ]:
create_table(df, model_order=list(gbiases.keys()))

## Robustness

In the paper [DaCy: A Unified Framework for Danish NLP](https://github.com/centre-for-humanities-computing/DaCy/blob/main/papers/DaCy-A-Unified-Framework-for-Danish-NLP/readme.md) we conducted a series on augmentation on the DaNE test set to estimate the robustness and biases of DaCy and other Danish language processing pipelines. This page represents only parts of the paper. We recommend reading the paper for a more thorough and nuanced overview.

The augmentation we will be using in this test are performed on the DaNE test set and include the following:

- **Spelling Error**: Intended to similar domains with inconsistent spelling, OCR errors, conversational data, etc.. The augmentation includes a series of smaller augmentation:
  - Keystroke error: The augmentation is used to introduce errors by replacing a character with a character that is close on the keyboard.
  - Character swap: The augmentation is used to introduce errors by swapping two neighboring characters.
  - Token swap: The augmentation is used to introduce errors by swapping two neighboring tokens.
- **Inconsistent Casing**: This augmentation is used to simulate inconsistent casing in the language and uses two different methods by either randomly capitalizing or lowercasing tokens.
- **Synonym Augmentation**: This augmentation is used to simulate the variation and slight grammatical errors in the language and uses two different methods:
  - Wordnet Synonym replacement: The augmentation replaces a token with a synonym in WordNet while respecting its syntactic role.
  - Embedding Synonym replacement: This augmentation replaces a token with a synonym which tends to appear in similar contexts.
- **Inconsistent Spacing**: This augmentation is used to simulate inconsistent spacing in the language and uses two different methods by either randomly adding or removing spaces.
- **Historical Spelling**: This augmentation is used to simulate historical spelling in Danish including ASCII spellings of the letters Æ (Ae), Ø (Oe), and Å (Aa) as well as uppercasing nouns.

For all of the augmentations the probability of an augmentation is set to 2% and the number of augmentations is set to 1. The augmentations are performed using the [augmenty](https://kennethenevoldsen.github.io/augmenty/index.html). 

Let's start by examining one of the augmentations:



````{admonition} Example

```{note} Original

Peter Schmeichel mener også, at det danske landshold anno 2021 tilhører verdenstoppen og kan vinde den kommende kamp mod England.
```

```{important} **2% simulated spelling errors**

ADD ME
```
````

The underlying assumption of making these augmentations is that the annotations of the tokens do not change with augmentation. This can naturally sometimes be the case. A single letter *"hun læste gåden"* (*she read the puzzle*) and *"hun løste gåden"* (*she solved the puzzle*) have quite a different meaning. So while we expect the performance to drop the degree of the drop is interesting to examine and often in comparison to the other models.

In [ ]:
robustness = {}
for mdl_name, model_getter in MODELS.items():
    if "fine_grained" in mdl_name:
        continue
    mdl_results = apply_models(
        mdl_name, model_getter, dataset="robustness_dane", splits=["test"]
    )
    robustness[mdl_name] = mdl_results

In [ ]:
tables = []
for mdl in robustness:
    print(mdl)
    examples = robustness[mdl]["test"]["examples"]

    aug_group = augmentation_specific_examples(examples)
    for aug_name, _examples in aug_group.items():
        _examples = convert_to_conll_2003(_examples) # also removes misc.
        table = create_dataframe(_examples, mdl, n_rep=100, n_samples=1000)
        table["Augmentation"] = aug_name
        tables.append(table)

## Inference Speed


While performance naturally is important is it also important to know why you might choose one model over another. One of the main reasons for choosing a smaller model is inference speed. The following table shows the inference speed of the different models. The inference speed is measured in words per second (WPS) and is measured on a Apple M1 Pro 16Gb running macOS 13.3.1 (i.e. high-end consumer laptop). The models are tested on the test set of DaNE.

```{adminition} GPU Acceleration
:class: note

These benchmarks does not use GPU acceleration. If you were to use GPU acceleration the inference speed would be much higher, similarly larger models would benefit more from this acceleration.
```

In [ ]:
dane = {}
for mdl_name, model_getter in MODELS.items():
    mdl_results = apply_models(mdl_name, model_getter, dataset="dane", splits=["test"])
    dane[mdl_name] = mdl_results["test"]

In [ ]:
rows = []
n_words = None
for mdl_name, model_getter in MODELS.items():
    total_time = dane[mdl_name]["test"]["time_in_seconds"]
    if n_words is None:
        examples = dane[mdl_name]["test"]["examples"]
        n_words = sum(len(e.y) for e in examples)
    wps = n_words / total_time
    rows.append({"Model": mdl_name, "Words per second": wps, "Total time": total_time})

speed = pd.DataFrame(rows)    

In [ ]:
speed

# testing


In [ ]:
from evaluation.datasets import datasets

ds = datasets.get("gender_bias_dane")()
examples1 = ds["test"]
ds = datasets.get("gender_bias_dane")()
examples2 = ds["test"]